# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 76 new papers today
          28 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - " +
                       "_" + paper['comments'] + "_")
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/27 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2209.11775


extracting tarball to tmp_2209.11775...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.11778


/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.11775/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


...

 done.
Retrieving document from  https://arxiv.org/e-print/2209.12850


/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.12814/rnaas.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


extracting tarball to tmp_2209.12850...

 done.


/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.14/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3378: LatexWarning: Found documentclass in tmp_2209.12850/aa.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.11781-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.11781) | **ALMA uncovers highly filamentary structure towards the Sgr E region**  |
|| J. Wallace, et al. -- incl., <mark>J. D. Henshaw</mark> |
|*Appeared on*| *2022-09-27*|
|*Comments*| *20 pages, 17 figures, accepted for publication in ApJ*|
|**Abstract**| We report on the discovery of linear filaments observed in CO(1-0) emission for a $\sim2'$ field of view toward the Sgr E star forming region centered at (l,b)=(358.720$^\circ$, 0.011$^\circ$). The Sgr E region is thought to be at the turbulent intersection of the ''far dust lane'' associated with the Galactic bar and the Central Molecular Zone (CMZ). This region is subject to strong accelerations which are generally thought to inhibit star formation, yet Sgr E contains a large number of HII regions. We present $^{12}$CO(1-0), $^{13}$CO(1-0), and C$^{18}$O(1-0) spectral line observations from ALMA and provide measurements of the physical and kinematic properties for two of the brightest filaments. These filaments have widths (FWHM) of $\sim0.1$ pc and are oriented nearly parallel to the Galactic plane, with angles from the Galactic plane of $\sim2^\circ$. The filaments are elongated, with lower limit aspect ratios of $\sim$5:1. For both filaments we detect two distinct velocity components that are separated by about 15 km s$^{-1}$. In the C$^{18}$O spectral line data with $\sim$0.09 pc spatial resolution, we find that these velocity components have relatively narrow ($\sim$1-2 km s$^{-1}$) FWHM linewidths when compared to other sources towards the Galactic center. The properties of these filaments suggest that the gas in the Sgr E complex is being ''stretched'' as it is rapidly accelerated by the gravitational field of the Galactic bar while falling towards the CMZ, a result that could provide insight into the extreme environment surrounding this region and the large-scale processes which fuel this environment. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.11775-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.11775) | **Millimeter Dust Emission and Planetary Dynamics in the HD 106906 System**  |
|| Anna Fehr, et al. -- incl., <mark>A. Meredith Hughes</mark> |
|*Appeared on*| *2022-09-27*|
|*Comments*| **|
|**Abstract**| Debris disks are dusty, optically thin structures around main sequence stars. HD 106906AB is a short-period stellar binary, host to a wide separation planet, HD 106906b, and a debris disk. Only a few known systems include a debris disk and a directly imaged planet, and HD 106906 is the only one in which the planet is exterior to the disk. The debris disk is edge-on and highly asymmetric in scattered light. Here we resolve the disk structure at a resolution of 0.38" (39 au) with the Atacama Large Millimeter/submillimeter Array (ALMA) at a wavelength of 1.3 mm. We model the disk with both a narrow and broad ring of material, and find that a radially broad, axisymmetric disk between radii of $\sim$50$-$100 au is able to capture the structure of the observations without evidence of any asymmetry or eccentricity, other than a tentative stellocentric offset. We place stringent upper limits on both the gas and dust content of a putative circumplanetary disk. We interpret the ALMA data in concert with scattered light observations of the inner ring and astrometric constraints on the planet's orbit, and find that the observations are consistent with a large-separation, low-eccentricity orbit for the planet. A dynamical analysis indicates that the central binary can efficiently stabilize planetesimal orbits interior to $\sim$100 au, which relaxes the constraints on eccentricity and semimajor axis somewhat. The observational constraints are consistent with in situ formation via gravitational instability, but cannot rule out a scattering event as the origin for HD 106906b's current orbit. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.11778-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.11778) | **Formation History of HD106906 and the Vertical Warping of Debris Disks  by an External Inclined Companion**  |
|| Nathaniel W. H. Moore, et al. -- incl., <mark>Gongjie Li</mark>, <mark>Lee Hassenzahl</mark> |
|*Appeared on*| *2022-09-27*|
|*Comments*| *13 pages, 7 figures. We are still in the process of responding to our referee, comments are welcome*|
|**Abstract**| HD106906 is a planetary system that hosts a wide-orbit companion, as well as an eccentric and flat debris disk, which hold important constraints on its formation and subsequent evolution. The recent observations of the companion constrain its orbit to be eccentric and inclined relative to the plane of the debris disk. Here, we show that, in the presence of the inclined companion, the debris disk quickly ($\lesssim5$ Myr) becomes warped and puffy. This suggests that the current configuration of the system is relatively recent. We explore the possibility that a recent close encounter with a free floating planet could produce a companion with orbital parameters that agree with observations of HD106906b. We find that this scenario is able to recreate the structure of the debris disk while producing a companion in agreement with observation. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.12850-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.12850) | **SIT 45: An interacting, compact, and star-forming isolated galaxy  triplet**  |
|| D. Grajales-Medina, et al. -- incl., <mark>U. Lisenfeld</mark> |
|*Appeared on*| *2022-09-27*|
|*Comments*| *13 pages, 7 figures, accepted for publication in Astronomy & Astrophysics*|
|**Abstract**| The merging system SIT 45 (UGC 12589) is an unusual isolated galaxy triplet, consisting of three merging late-type galaxies, out of 315 systems in the SIT (SDSS-based catalogue of Isolated Triplets). The main aims of this work are to study its dynamical evolution and star formation history (SFH), as well as its dependence on its local and large-scale environment. To study its dynamics, parameters such as the velocity dispersion ($\sigma_{v}$), the harmonic radius ($R_{H}$), the crossing time ($H_0t_c$), and the virial mass ($M_{vir}$), along with the compactness of the triplet ($S$) were considered. To constrain the SFH, we used CIGALE to fit its observed spectral energy distribution using multi-wavelength data from the ultraviolet to the infrared. According to its SFH, SIT 45 presents star-formation, where the galaxies also present recent ($\sim $200 Myr) star-formation increase, indicating that this activity may have been triggered by the interaction. Its dynamical configuration suggests that the system is highly evolved in comparison to the SIT. However this is not expected for systems composed of star-forming late-type galaxies, based on observations in compact groups. We conclude that SIT 45 is a system of three interacting galaxies that are evolving within the same dark matter halo, where its compact configuration is a consequence of the on-going interaction, rather than due to a long-term evolution (as suggested from its $H_0t_c$ value). We consider two scenarios for the present configuration of the triplet, one where one of the members is a tidal galaxy, and another where this galaxy arrives to the system after the interaction. Both scenarios need further exploration. The isolated triplet SIT 45 is therefore an ideal system to study short timescale mechanisms ($\sim 10^8$ years), such as starbursts triggered by interactions which are more frequent at higher redshift. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.12422-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.12422) | **Six More Ultra-Faint Milky Way Companions Discovered in the DECam Local  Volume Exploration Survey**  |
|| W. Cerny, et al. -- incl., <mark>C. E. Martínez-Vázquez</mark>, <mark>T. S. Li</mark> |
|*Appeared on*| *2022-09-27*|
|*Comments*| *30 pages, 12 Figures (including Appendix). Submitted to ApJ. We encourage the reader to also review Smith et al. 2022, "Discovery of a new Local Group Dwarf Galaxy Candidate in UNIONS: Bo\"{o}tes V" (arxiv: 2209.08242), who independently present the discovery of one of the candidates reported here. We are working to make code and data products available*|
|**Abstract**| We report the discovery of six ultra-faint Milky Way satellites discovered through matched-filter searches conducted using Dark Energy Camera (DECam) data processed as part of the second data release of the DECam Local Volume Exploration (DELVE) survey. Leveraging deep Gemini/GMOS-N imaging (for four candidates) as well as follow-up DECam imaging (for two candidates), we characterize the morphologies and stellar populations of these systems. We find that these candidates all share faint absolute magnitudes ($M_{V} \geq -3.2$ mag) and old, metal-poor stellar populations ($\tau > 10$ Gyr, [Fe/H] $< -1.4$ dex). Three of these systems are more extended ($r_{1/2} > 15$ pc), while the other three are compact ($r_{1/2} < 10$ pc). From these properties, we infer that the former three systems (Bo\"{o}tes V, Leo Minor I, and Virgo II) are consistent with ultra-faint dwarf galaxy classifications, whereas the latter three (DELVE 3, DELVE 4, and DELVE 5) are likely ultra-faint star clusters. Using data from the Gaia satellite, we confidently measure the proper motion of Bo\"{o}tes V, Leo Minor I, and DELVE 4, and tentatively detect a proper motion signal from DELVE 3 and DELVE 5; no signal is detected for Virgo II. We use these measurements to explore possible associations between the newly-discovered systems and the Sagittarius dwarf spheroidal, the Magellanic Clouds, and the Vast Polar Structure, finding several plausible associations. Our results offer a preview of the numerous ultra-faint stellar systems that will soon be discovered by the Vera C. Rubin Observatory and highlight the challenges of classifying the faintest stellar systems. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Planck' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.11873-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.11873) | **After DART: Using the first full-scale test of a kinetic impactor to  inform a future planetary defense mission**  |
|| Thomas S. Statler, et al. -- incl., <mark>Mallory E. DeCoster</mark>, <mark>Harrison F. Agrusa</mark>, <mark>Alain Herique</mark>, <mark>Isabel Herreros</mark> |
|*Appeared on*| *2022-09-27*|
|*Comments*| *30 pages, 7 figures. Planetary Science Journal, in press, accepted 2022 September 22*|
|**Abstract**| NASA's Double Asteroid Redirection Test (DART) is the first full-scale test of an asteroid deflection technology. Results from the hypervelocity kinetic impact and Earth-based observations, coupled with LICIACube and the later Hera mission, will result in measurement of the momentum transfer efficiency accurate to ~10% and characterization of the Didymos binary system. But DART is a single experiment; how could these results be used in a future planetary defense necessity involving a different asteroid? We examine what aspects of Dimorphos's response to kinetic impact will be constrained by DART results; how these constraints will help refine knowledge of the physical properties of asteroidal materials and predictive power of impact simulations; what information about a potential Earth impactor could be acquired before a deflection effort; and how design of a deflection mission should be informed by this understanding. We generalize the momentum enhancement factor $\beta$, showing that a particular direction-specific $\beta$ will be directly determined by the DART results, and that a related direction-specific $\beta$ is a figure of merit for a kinetic impact mission. The DART $\beta$ determination constrains the ejecta momentum vector, which, with hydrodynamic simulations, constrains the physical properties of Dimorphos's near-surface. In a hypothetical planetary defense exigency, extrapolating these constraints to a newly discovered asteroid will require Earth-based observations and benefit from in-situ reconnaissance. We show representative predictions for momentum transfer based on different levels of reconnaissance and discuss strategic targeting to optimize the deflection and reduce the risk of a counterproductive deflection in the wrong direction. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.12060-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.12060) | **The Sky Distribution and Magnitudes of Starlink Satellites by the Year  2027**  |
|| <mark>Anthony Mallama</mark> |
|*Appeared on*| *2022-09-27*|
|*Comments*| **|
|**Abstract**| Visual magnitudes and sky coordinates are projected for the full constellation of Starlink satellites. The results are presented in the form of sky maps and numerical tables. Observer latitudes from the equator to 60 degrees are considered. The solar elevations include -12 deg (the end of nautical twilight), -18 deg (the end of astronomical twilight) and -30 deg. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2209.12199-b31b1b.svg)](https://arxiv.org/abs/arXiv:2209.12199) | **RC100: Rotation Curves of 100 Massive Star-Forming Galaxies at z=0.6-2.5  Reveal Little Dark Matter on Galactic Scales**  |
|| A. Nestor Shachar, et al. -- incl., <mark>N.M. Förster Schreiber</mark>, <mark>R.I. Davies</mark>, <mark>R. Herrera-Camus</mark>, <mark>L. L. Lee</mark>, <mark>D. Liu</mark> |
|*Appeared on*| *2022-09-27*|
|*Comments*| *Submitted to ApJ (34 pages, 12 figures, 4 tables)*|
|**Abstract**| We analyze Ha or CO rotation curves (RCs) extending out to several galaxy effective radii for 100 massive, large, star-forming disk galaxies (SFGs) across the peak of cosmic galaxy star formation (z~0.6-2.5), more than doubling the previous sample presented by Genzel et al. (2020) and Price et al. (2021). The observations were taken with SINFONI and KMOS integral-field spectrographs at ESO-VLT, LUCI at LBT, NOEMA at IRAM, and ALMA. We fit the major axis kinematics with beam-convolved, forward models of turbulent rotating disks with bulges embedded in dark matter (DM) halos, including the effects of pressure support. The fraction of dark to total matter within the disk effective radius ($R_e ~ 5 kpc$), $f_DM (R_e)=V_{DM}^2 (R_e)/V_{circ}^2 (R_e)$, decreases with redshift: At z~1 (z~2) the median DM fraction is $0.38\pm 0.23$ ($0.27\pm 0.18$), and a third (half) of all galaxies are "maximal" disks with $f_{DM} (R_e)<0.28$. Dark matter fractions correlate inversely with the baryonic surface density, and the low DM fractions require a flattened, or cored, inner DM density distribution. At z~2 there is ~40% less dark matter mass on average within $R_e$ compared to expected values based on cosmological stellar-mass halo-mass relations. The DM deficit is more evident at high star formation rate (SFR) surface densities ($\Sigma_{SFR}>2.5 M_{\odot} yr^{-1} kpc^{-2}$) and galaxies with massive bulges ($M_{bulge}>10^{10} M_{\odot}$). A combination of stellar or active galactic nucleus (AGN) feedback, and/or heating due to dynamical friction, either from satellite accretion or clump migration, may drive the DM from cuspy into cored mass distributions. The observations plausibly indicate an efficient build-up of massive bulges and central black holes at z~2 SFGs. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error not a gzip file</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2209.11781.md
    + _build/html/tmp_2209.11781/./mini_mom0_maps_13CO.png
    + _build/html/tmp_2209.11781/./Moment0_12CO_masked.png
    + _build/html/tmp_2209.11781/./Moment0_13CO_masked.png
    + _build/html/tmp_2209.11781/./Moment0_C18O_masked.png
    + _build/html/tmp_2209.11781/./pv1_12.png
    + _build/html/tmp_2209.11781/./pv1_13.png
    + _build/html/tmp_2209.11781/./pv_1_18.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{$\vdag$}{(v)^\dagger}$
$\newcommand$
$\newcommand$</div>



<div id="title">

# ALMA uncovers highly filamentary structure towards the Sgr E region

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2209.11781-b31b1b.svg)](https://arxiv.org/abs/2209.11781)<mark>Appeared on: 2022-09-27</mark> - _20 pages, 17 figures, accepted for publication in ApJ_

</div>
<div id="authors">

J. Wallace, et al. -- incl., <mark>J. D. Henshaw</mark>

</div>
<div id="abstract">

**Abstract:** We report on the discovery of linear filaments observed in CO(1-0) emission for a$\sim2$\arcmin$$field of view toward the Sgr E star forming region centered at$(l,b)=(358.720\degree, 0.011\degree$). The Sgr E region is thought to be at the turbulent intersection of the "far dust lane" associated with the Galactic bar and the Central Molecular Zone (CMZ). This region is subject to strong accelerations which are generally thought to inhibit star formation, yet Sgr E contains a large number of HII regions.  We present$^{12}$CO(1-0),$^{13}$CO(1-0), and C$^{18}$O(1-0) spectral line observations from ALMA and provide measurements of the physical and kinematic properties for two of the brightest filaments. These filaments have widths (FWHM) of$\sim0.1$pc and are oriented nearly parallel to the Galactic plane, with angles from the Galactic plane of$\sim2\degree$. The filaments are elongated, with lower limit aspect ratios of$\sim$5:1. For both filaments we detect two distinct velocity components that are separated by about 15 km s$^{-1}$. In the C$^{18}$O spectral line data with$\sim$0.09 pc spatial resolution,  we find that these velocity components have relatively narrow ($\sim$1-2 km s$^{-1}$) FWHM linewidths when compared to other sources towards the Galactic center. The properties of these filaments suggest that the gas in the Sgr E complex is being "stretched" as it is rapidly accelerated by the gravitational field of the Galactic bar while falling towards the CMZ, a result that could provide insight into the extreme environment surrounding this region and the large-scale processes which fuel this environment.

</div>

<div id="div_fig1">

<img src="tmp_2209.11781/./mini_mom0_maps_13CO.png" alt="Fig12" width="100%"/>

**Figure 12. -** Integrated intensity (moment 0) maps from the $^{13}$CO data cube. Each image shows the integrated intensity over a range of 5 km s$^{-1}$. The specific velocity ranges are indicated above each image. The top, orange contour indicates the region we define for filament 1, while the bottom, blue contour indicates the filament 2 region. The beam is indicated in the lower right corner of each channel map.  (*fig:mini_mom0*)

</div>
<div id="div_fig2">

<img src="tmp_2209.11781/./Moment0_12CO_masked.png" alt="Fig3.1" width="33%"/><img src="tmp_2209.11781/./Moment0_13CO_masked.png" alt="Fig3.2" width="33%"/><img src="tmp_2209.11781/./Moment0_C18O_masked.png" alt="Fig3.3" width="33%"/>

**Figure 3. -** Spatially masked moment 0 maps generated from data in the velocity range of -197 km s$^{-1}$ to -231 km s$^{-1}$ for $^{12}$CO (\textit{top}), $^{13}$CO (\textit{middle}), and C$^{18}$O (\textit{bottom}). In the $^{12}$CO image, the orange line represents the filament 1 \textit{p-v} slice path,  while the blue line represents the filament 2 \textit{p-v} slice path. For each image, the beam is indicated in the lower right corner. (*fig:mom0*)

</div>
<div id="div_fig3">

<img src="tmp_2209.11781/./pv1_12.png" alt="Fig6.1" width="33%"/><img src="tmp_2209.11781/./pv1_13.png" alt="Fig6.2" width="33%"/><img src="tmp_2209.11781/./pv_1_18.png" alt="Fig6.3" width="33%"/>

**Figure 6. -** The filament 1 position-velocity diagrams for $^{12}$CO (\textit{top)}, $^{13}$CO (\textit{middle}), and  C$^{18}$O (\textit{bottom)}, respectively. The $p-v$ slice starts on the end of the filament that is closer to the Galactic center, so the offset represents the distance along the $p-v$ slice as we move from left to right along the filament 1 spine (see Figure \ref{fig:mom0}). (*fig:PV_fil_1*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

45  publications in the last 7 days.
	 _build/html/2209.11781.md
	 _build/html/2209.11602.md
	 _build/html/2209.11248.md
	 _build/html/2209.11096.md
	 _build/html/2209.10618.md
	 _build/html/2209.09914.md
	 _build/html/2209.09823.md
	 _build/html/2209.08242.md
	 _build/html/2209.08120.md
	 _build/html/2209.08106.md
	 _build/html/2209.07962.md
	 _build/html/2209.07931.md
	 _build/html/2209.06833.md
	 _build/html/2209.05814.md
	 _build/html/2209.05593.md
	 _build/html/2209.04304.md
	 _build/html/2209.03367.md
	 _build/html/2209.02872.md
	 _build/html/2209.02725.md
	 _build/html/2209.02722.md
	 _build/html/2209.02092.md
	 _build/html/2209.01125.md
	 _build/html/2209.00828.md
	 _build/html/2209.00363.md
	 _build/html/2208.14927.md
	 _build/html/2208.09335.md
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/ht

In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers